In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define example values for n_timesteps and n_features
n_timesteps = 10  # Number of time steps
n_features = 5    # Number of features

# Define the model
model = Sequential()

# Add the first LSTM layer with 50 units, returning sequences
model.add(LSTM(50, return_sequences=True, input_shape=(n_timesteps, n_features)))


In [ ]:
import pandas as pd
import numpy as np
import datetime

# Generate synthetic data
date_rng = pd.date_range(start='1/1/2020', end='1/1/2021', freq='D')
product_ids = ['P001', 'P002', 'P003']
data = []

for product_id in product_ids:
    for date in date_rng:
        sales = np.random.randint(1, 20)
        promotions = np.random.choice([0, 1])
        economic_index = np.random.uniform(0.8, 1.2)
        seasonality_index = np.random.uniform(0.9, 1.1)
        stock_level = np.random.randint(50, 200)
        reorder_point = np.random.randint(20, 50)
        lead_time = np.random.randint(1, 7)
        data.append([date, product_id, sales, promotions, economic_index, seasonality_index, stock_level, reorder_point, lead_time])

df = pd.DataFrame(data, columns=['date', 'product_id', 'sales', 'promotions', 'economic_index', 'seasonality_index', 'stock_level', 'reorder_point', 'lead_time'])
df.to_csv('inventory_data.csv', index=False)



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Load dataset
df = pd.read_csv('inventory_data.csv')

# Check for missing values and handle them if necessary
print(df.isnull().sum())

# Filter data for product P001
df_product = df[df['product_id'] == 'P001']

# Normalize features
feature_columns = ['sales', 'promotions', 'economic_index', 'seasonality_index', 'stock_level', 'reorder_point', 'lead_time']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_product[feature_columns])

# Normalize target variable separately
target_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_target = target_scaler.fit_transform(df_product[['sales']])

# Combine the scaled features and target
scaled_data_combined = np.hstack((scaled_target, scaled_data[:, 1:]))

# Create time series sequences
def create_sequences(data, time_steps=30):
    sequences = []
    labels = []
    for i in range(len(data) - time_steps):
        sequences.append(data[i:i+time_steps])
        labels.append(data[i+time_steps][0])  # Assuming the first column is the target variable
    return np.array(sequences), np.array(labels)

time_steps = 30
X, y = create_sequences(scaled_data_combined, time_steps)

# Check shapes of X and y
print(f'X shape: {X.shape}, y shape: {y.shape}')

# Split into training and testing data
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Define the model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Train the model
model.fit(X_train, y_train, batch_size=1, epochs=10)

# Predict future demand
future_sequence = scaled_data_combined[-time_steps:]
future_sequence = future_sequence.reshape((1, time_steps, future_sequence.shape[1]))
predicted_demand = model.predict(future_sequence)

# Inverse transform the prediction using the target scaler
predicted_demand = target_scaler.inverse_transform(predicted_demand)

print(f"Predicted inventory demand for the next period: {predicted_demand[0][0]}")


date                 0
product_id           0
sales                0
promotions           0
economic_index       0
seasonality_index    0
stock_level          0
reorder_point        0
lead_time            0
dtype: int64
X shape: (337, 30, 7), y shape: (337,)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 30, 50)            11600     
                                                                 
 lstm_6 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26  